# Setup

In [1]:
# import common libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import specific components from scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, RocCurveDisplay

# enhanced stats functions
from scipy import stats

# for ease of data profiling
from pandas_profiling import ProfileReport

In [2]:
# print environment setup details
print(f"pandas version  : {pd.__version__}")  # 1.4.3
print(f"numpy version   : {np.__version__}")  # 1.23.1
print(f"seaborn version : {sns.__version__}") # 0.11.2

pandas version  : 1.4.3
numpy version   : 1.23.2
seaborn version : 0.11.2


In [3]:
# set the number of columns for display so we can see all of them
pd.set_option('display.max_columns', 500)

Here, we also set the seed for numpy's random number generator such that our results are fully reproducible. This is because the other libraries (e.g. scikit-learn) use this random number generator, so if we set the seed we will always generate the same random numbers in the same sequence.

Thus, whenever we run the notebook from top-to-bottom, we will end up with the *exact* same results! 

In [4]:
SEED = 123
np.random.seed(SEED)

# Data Loading
To load in the data for this project, read in `bank-additional-full.csv` into a variable called `df` as a pandas DataFrame. The first two rows of the DataFrame should look like this:

|     | age | job | marital | education | default | housing | loan | contact | month | day_of_week | duration | campaign | pdays | previous | poutcome | emp.var.rate | cons.price.idx | cons.conf.idx | euribor3m | nr.employed | y |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **0** | 56 | housemaid | married | basic.4y | no | no | no | telephone | may | mon | 261 | 1 | 999 | 0 | nonexistent | 1.1 | 93.994 | -36.4 | 4.857 | 5191.0 | no |
| **1** | 57 | services | married | high.school | unknown | no | no | telephone | may | mon | 149 | 1 | 999 | 0 | nonexistent | 1.1 | 93.994 | -36.4 | 4.857 | 5191.0 | no |

Note that the file may not be using a conventional delimiter (e.g. comma, tabs, etc.). Figure out what the delimiter is and read the file accordingly :) 

In [7]:
# read in data
df = pd.read_csv('bank-additional-full.csv',sep = ';')

In [8]:
# make sure data was read in properly and matches above table
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# EDA
Now it is time for you to practice your EDA skills! Take a look at the Quest 3 Quiz, and answer the first 4 questions on the results of your EDA on the dataset.

This time, you don't get the data dictionary! You have to go look at the dataset's website and take a look yourself, like you would have to if this was a real project.

Here's the website where the dataset came from: [the UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/bank+marketing).

In [9]:
# optional: use pandas-profiling to do quick first pass at EDA
profile = ProfileReport(df, title="Bank Marketing Dataset Profiling Report")
# create html file to view report
profile.to_file("bank_marketing_dataset_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# conduct any other EDA that you need to in order to get a good feel for the data
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


With the EDA you have conducted, answer the following questions from the quiz. Note that these questions do not cover everything you should be looking for when doing EDA, they are just to give you an idea of what EDA would look like on such a dataset.
## Question 1
What percentage of users ended up subscribing to a deposit? (i.e. column `y` == 'yes') Round your answer to 1 decimal place.

In [ ]:
# calculate percentage of users where y == "yes"


This is interesting, it seems the vast majority of users did not subscribe to a deposit. This is very important to note - we have what's called an "imbalanced classification" problem - i.e. we do not have balance in the classes that we are classifying. This affects how we will handle our modelling later, and we'll revisit this. For now, just keep this fact in mind.

## Question 2
How many duplicate rows are in the dataset? (i.e. fewest number of rows you would have to remove in order to be left without duplicate rows)

In [ ]:
# determine number of duplicate rows


## Question 3
What does a `pdays` value of 999 mean?

(hint: you'll have to look at the dataset website!)

## Question 4
Are there any high correlations (e.g. ±0.5) between independent variables? If yes, how high?

In [ ]:
# plot correlation heatmap to get a sense of what variables are correlated


Wow, that's problematic. There will very likely be multicollinearity issues if we don't deal with that when wrangling our data. Let's deal with the data wrangling now.

# Data Wrangling
As we did before in Quest 2, once we've adequately explored our data, we need to wrangle it for modelling.

In this case, we also don't have any missing data, but we do have several duplicate rows, a lot of independent variables correlated with one another, and a LOT of categorical variables! 

One step at a time. First, let's deal with the duplicate rows.

In [ ]:
# drop duplicates in-place, and reset the index so no index values are skipped


In [ ]:
# test to make sure nothing left is duplicated
assert df.duplicated().any() == False, "Some duplicates remain."

Now, let's drop some variables. If you read closely, the dataset instructs us to discard the `duration` variable "if the intention is to have a realistic predictive model", which ours is.

If you don't know where that came from, go back and check the dataset webpage again. This is an important practice to note!

Drop the following variables from df:
* duration
* emp.var.rate
* cons.price.idx
* cons.conf.idx
* euribor3m
* nr.employed

In [ ]:
# drop variables from df


In [ ]:
print(f"variables after dropping: {len(df.columns)}")

Now, get dummy variables for all our categorical variables using `pd.get_dummies()`, ensuring to use the `drop_first=True` argument to mitigate possible multicollinearity issues.

In [ ]:
# get dummies for all categorical variables


## Question 5
How many variables were left after dropping variables, and after getting dummies?

In [ ]:
print(f"variables after getting dummies: {len(df.columns)}")

Now verify that we still don't have major multicollinearity issues. We have a few too many variables to use the heatmap directly, so you'll have to be clever and try to figure out another way of doing it.

Here's a hint - previously, we called `df.corr()` which gave us the pair-wise correlations of each variable, and then plotted it using the heatmap. Maybe we can start with `df.corr()` and do something else to find highly correlated variables? Something that may help is the `DataFrame.itertuples()` method - [documentation here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.itertuples.html).

(If you can't figure it out, I put one possible solution all the way at the bottom of the notebook. But for the purposes of learning, I suggest you try and struggle at this problem for at least a little while first!)

In [ ]:
# get list of all variable combinations from highest correlation to lowest
df.corr()

Regardless, there are still a number of variables which are still strongly correlated to each other. In fact, there are some which are *exactly* identical to one another! So we'll remove a few more variables:
* loan_unknown
* poutcome_success
* poutcome_nonexistent

In theory, a better approach would likely be to remove variables depending on whether our model improves or not when they are removed. For the sake of simplicity, we're doing this analysis just from data to model, but normally your model's performance would feedback on the choices of your data handling/variable selection/feature engineering, and you'd adjust accordingly.

In [ ]:
# drop these 3 other variables from df


While we're at it, the column named `y_yes` is a bit silly. Let's rename it to something more informative, like `deposit`.

In [ ]:
# rename "y_yes" column to "deposit"


Excellent! Now separate our independent variables into a variable called `X`, and our target variable `deposit` into a variable called `y`.

In [ ]:
# split up our variables
X = 
y = 

In [ ]:
# the resulting shapes of X and y should be (41176, 44) and (41176,) respectively
print(X.shape)
print(y.shape)

Now we need to split up our data into training and test data. Using scikit-learn's `train_test_split` function, using a `test_size` of 0.3 (i.e. 30% of data in test set), and ensure that the random state is set to our seed from above.

Documentation for `train_test_split()` can be found here: https://scikit-learn.org/1.1/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# do the train test split
X_train, X_test, y_train, y_test = 

Now let's verify to make sure you split your data identically. 

In [ ]:
print("expected : 29491, 26771, 22437, 6760, 28406")
print("actual   : " + ", ".join([str(x) for x in X_train.head().index]))

Now, we're going to do something that we didn't do in Quest 2 (because I didn't want to introduce too many concepts at once), but is a machine learning best practice - standardizing your dataset!

Essentially, if some numeric variables are much bigger than the others (e.g. age goes up to the 90s, whereas our one-hot encoded dummy variables are only 0 or 1), they may have a larger impact on the final model. As such, we scale all our variables so comparisons between them can be made fairly.

Scikit-learn provides a helpful utility for this, `StandardScaler()` - its [documentation is here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). 

### Optional
If you go back and scale your features in Quest 2, does the model perform better with or without scaling? 

In [ ]:
# fit scaler
scaler = 

In [ ]:
# transform X_train and X_test
X_train = 
X_test = 

In [ ]:
# visualize results
X_train[0]

If you see an array which is a lot of numbers generally ranging from -2 to +2, then your scaling should have worked! Nicely done.

# Modelling and Evaluation
Now, you might remember that our dataset is an imbalanced dataset, with far more values of `0` for deposit than `1`. If we don't properly handle this, we're going to have trouble! For example, your model could end up with 90% accuracy if it just said that every single value was `0`, which is not something that we want.

Just to name a few ways of solving this problem (this is not an exhaustive list!):
1. Sub-sample the `0`s, throwing away most of the data until we have an even split of `0` and `1`.
2. Over-sample the `1`s, duplicating much of the `1` data until we have an even split.
3. Weight the model towards the minority class (`1` in this case) so that it places extra emphasis on it while learning.

Each of these approaches has different pros and cons. For this quest, we'll be doing approach 3, which scikit-learn has a convenient parameter for. 

So now let's make our model! Check the docs for the `LogisticRegression()` scikit-learn function to figure out what the `class_weight` parameter does, then make and fit the training data to your model.

Documentation for `LogisticRegression()` can be found here: https://scikit-learn.org/1.1/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
# instantiate logistic regression model and fit the training data to it
lr = 

## Question 6
For your trained logistic regression model, what is the intercept and the coefficient for the `age` feature?

Hint: check the LogisticRegression() documentation to see how to find these attributes!

In [ ]:
# describe model


Now we need to evaluate how our model did. Let's get the score, classification report, confusion matrix, and ROC curve, just as was done in the lesson video.

### Training set

In [ ]:
# score on training set


In [ ]:
# predict training set values
predictions_train = 
# classification report on training set


In [ ]:
# plot confusion matrix for training set


Note that the `plot_roc_curve()` function used in the video is actually deprecated, and it is recommended to use either `RocCurveDisplay.from_estimator()` or `RocCurveDisplay.from_predictions()` now.

See the documentation for further details: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html.

In [ ]:
# use RocCurveDisplay to plot a ROC curve on training set
RocCurveDisplay.from_estimator(lr, X_train, y_train)

Wow, those results aren't impressive at all. Our ability to make positive predictions is somewhat underwhelming. But, of course, perhaps that truly is the best we can do with the data we used. Let's move on and evaluate the same metrics on the test set first, before we draw any rash conclusions.

### Test set

In [ ]:
# score on test set


In [ ]:
# predict test set values

# classification report on test set


In [ ]:
# plot confusion matrix for test set


In [ ]:
# use RocCurveDisplay to plot a ROC curve on test set


# Discussion
While our model has clearly learned something about the data, since it's performing better than chance, there is still a lot of room for improvement here.

But before you get too demoralised - this is a much more realistic example of what it looks like when you model real data! Real data is messy, with a lot of complex interactions that are hard to anticipate when first beginning a modelling exercise. As a result, it's rare that your first attempt to model a dataset works out perfectly. Modelling is often an iterative process, with the results from each round informing subsequent modelling attempts. 

Also, data on human behaviour is often messy - it's hard to precisely predict humans, given how complex our motives and justifications for our actions often are.

So, honestly, this is a pretty realistic first attempt at modelling! What could we try in subsequent models? Here's a non-exhaustive list of ideas:
1. Our feature selection process was very ad hoc - perhaps we should try selecting features more deliberately. We might have left out some valuable information when throwing out variables, or we may have left in redundant/unnecessary features!
2. Additional feature engineering - we haven't considered any possible interaction effects. Check out [this resource for more information on interaction effects](https://stattrek.com/multiple-regression/interaction).
3. Also on feature engineering, we could try modifying our input features. Instead of having one variable per job, maybe we could group them by approximate income level. We could try using weekdays and weekends instead of just `day_of_week`. Separating months into seasons could also help. You don't know until you try!
4. Maybe we should standardise/normalise our data in other ways. For instance, we didn't handle the special case of `pdays==999`. We could probably handle this in several different ways - for example, replacing those values with the highest non-999 value of `pdays`, omitting it entirely (i.e. making it a `NaN`), create a separate dummy variable to indicate it, etc.
5. There are other ways of tackling the unbalanced classification problem. Maybe if we tried those, our model would perform better?

As with Quest 2, attempting further improvements are left as an exercise to the reader ;)

# Prepare your Submission
Congratulations, you made it to the end of this lesson! Be sure to complete the few remaining quiz questions, and prepare your submission:
1. Replace the value of the NAME variable with your StackUp name.
2. Edit the last cell in the notebook so that it reflects the requested data, rather than the placeholder values below. Make sure you read what it is asking for, and replicate all the features of it - this includes axis labels, which charts go where, etc. 
3. Restart your notebook, then run it from start to end.
4. Take a screenshot of the output from the last cell in this notebook. Make sure your screenshot shows your taskbar/dock (i.e. take a screenshot of the whole screen, not just the window).

e.g. This section of code currently produces placeholder values:  

```print(f"{'intercept':<20}: {np.random.random():.4f}")```

You must replace `np.random.random()` so that it reflects the intercept of your model.

In [ ]:
# replace this with your stackup name
NAME = "yourstackupname"

In [ ]:
# fill out this cell appropriately, then take a screenshot of the result
print(f"{'Name':<20}: {NAME}")
# describe model
print(f"{'intercept':<20}: {np.random.random():.4f}")
print("5 coefficients with largest absolute value:")
a = sorted([np.random.random()*2-1 for _ in range(5)], key=lambda x: abs(x), reverse=True)
for i, val in enumerate(a):
    print(f"{df.columns[np.random.randint(0,len(df.columns))]:<20}: {val:.4f}")
print("---"*8)
print(f"{'Score on test set':<20}: {np.random.random():.4f}")
print(f"{'Plots for test set':<20}:")
# draw plots for test set
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(10,5)

# ax1 - confusion matrix
sns.heatmap(np.random.randint(100,1000,(2,2)), ax=ax1)

# ax2 - draw RocCurveDisplay
ax2.axline([0, 0], [1, 1])

plt.show()